# Contents
1. [Generate a vcf file with structural variations from the dna181_illumina.bam file](#Generate a vcf file with structural variations from the dna181_illumina.bam file)
2. [Identification of de novo structural variations](#Identification of de novo structural variations)
3. [Identification of copy number variations in dna181_illumina.bam file](#Identification of copy number variations in dna181_illumina.bam file)
4. [Annotation of structural variations using gene overlap](#Annotation of structural variations using gene overlap)

Identification of structural variations and copy number variations in NGS data.
==================================================================

In this exercise you will learn how to evaluate NGS data for the presence of structural variations and copy number changes. The starting point for detecting these variations will be a BAM file (Binary Alignment Map) as you have worked with during COO1 and COO2. 

We have generated a partial whole-genome sequencing dataset for a patient with developmental malformations. The data set consists of paired-end Illumina sequencing data (2 x 150bp reads). You are going to find out if the patient genome contains perhaps structural variations that could underlie the disease phenotype. 

The data set can be found here: datafiles/COO3/dna181_illumina.bam


1. Generate a vcf file with structural variations from the dna181_illumina.bam file<a name="Generate a vcf file with structural variations from the dna181_illumina.bam file"></a>
=============
Manta (https://github.com/Illumina/manta) is a bioinformatic tool that enables the processing of read alignments in BAM files and identify candidate structural variations.


<span style="color:green">[DO]: Run manta on the dna181_illumina.bam file, to generate a vcf output file with detected structural variations.</span>

In [ ]:
%%bash
/home/ubuntu/external/tools/manta_1.0.3/bin/configManta.py \
--bam datafiles/COO3/dna181_illumina.bam \
--referenceFasta Homo_sapiens.GRCh37.GATK.illumina/Homo_sapiens.GRCh37.GATK.illumina.fasta \
--runDir datafiles/COO3/manta/


[NOTE]: The next step will take approximately 5 minutes.

In [ ]:
%%bash
datafiles/COO3/manta/runWorkflow.py -m local -j 2 -g 6

In [ ]:
%%bash
gunzip -c datafiles/COO3/manta/results/variants/diploidSV.vcf.gz > datafiles/COO3/dna181_manta.vcf

Now you have a VCF file with structural variation calls. Open the VCF file and take a look at its contents. 

<span style="color:purple">[Q]: Take a look at the first non-header line of the VCF file. Answer the following questions:
-	<span style="color:purple">What does one non-header line represent exactly?
-	<span style="color:purple">What are the start and end coordinates of the variation in the first non-header line?
-	<span style="color:purple">What is the orientation of the variation in the first non-header line?
-	<span style="color:purple">Is this variation homozygous or heterozygous?
</span>

[A]:

<span style="color:purple">Calculate the number of interchromosomal heterozygous variations in the VCF file.<BR>
Write your own python script:

In [ ]:
[A]:

<span style="color:purple">or use SNPsift:

In [ ]:
%%bash
export SNPSIFT=/home/ubuntu/external/tools/snpEff/SnpSift.jar


2. Identification of de novo structural variations<a name="Identification of de novo structural variations"></a>
=============
In a second step we will check if the structural variations identified in step 1 are de novo or whether they are inherited from the parents of this patient. Therefore we have created a multi-sample vcf file for you based on the BAM files from the father, mother and the child/patient using the Manta command: `manta/bin/configManta.py --bam child.bam --bam father.bam --bam mother.bam --referenceFasta hg19.fa --runDir outdir`
and the execution command:
`outdir/runWorkflow.py -m local -j 8`

<span style="color:purple">[Q]: Read the vcf file `datafiles/COO3/dna181_illumina_trioSV.vcf`. Answer the following questions:
-	<span style="color:purple">What are the genotypes of the first variation in the child (C), father (F) and mother (M)?
-	<span style="color:purple">Is this variant de novo or inherited?
-	<span style="color:purple">What is the amount of non-reference (alt) reads in the child for split reads (SR) and paired reads (PR)?


[A]:

<span style="color:purple">[Q]: Write a python script to parse out the de novo variations in the child. How many are there?

In [ ]:
[A]:

3. Identification of copy number variations in dna181_illumina.bam file<a name="Identification of copy number variations in dna181_illumina.bam file"></a>
=============
There are different methods to detect structural variations in NGS data. Manta uses paired-end and split-read mappings to identify structural variations. However, changes in DNA copy number (CNVs) are not detected by Manta. Therefore, we will run another tool – FREEC (https://github.com/BoevaLab/FREEC) - to detect CNVs.

<span style="color:purple">[Q]: What type of variations cannot be detected by FREEC?

[A]:

<span style="color:green">[DO]: Run FREEC on the dna181_illumina.bam file. This step will take approximately 5 minutes.

In [ ]:
%%bash
freec -conf datafiles/COO3/freec_config.txt

The output of FREEC is a list of predicted changes in DNA copy number. In the file `datafiles/COO3/freec/dna181_illumina.bam_CNVs` you can find the predicted CNVs.

<span style="color:purple">[Q]: What do the two coordinates for each variant indicate? 


[A]:

<span style="color:purple">[Q]: What is the largest CNV in the list? Please ignore the first and last CNV for each chromosome, because these represent artifacts of the calling since only a subset of the data was used.  

In [ ]:
[A]:

Now you are going to check if any of the CNVs matches with a structural variation breakpoint in the vcf file that you have generated above. There are several steps needed to get to this answer.

<span style="color:green">[DO]: Generate a bed file with a list of each breakpoint coordinate of the de novo SVs (note: every SV has two breakpoints!). List them in bed format: chr [TAB] coordinate [TAB] coordinate

In [ ]:
[A]:    

<span style="color:green">[DO]: Do the same for the *_CNV file with CNV predictions.

In [ ]:
[A]:

<span style="color:green">[DO]: Check the overlap of the breakpoint bed file with the CNV bed file using bedtools: ‘bedtools closest fileA fileB’.

In [ ]:
%%bash
bedtools sort -i datafiles/COO3/sv.bed > datafiles/COO3/sv.sorted.bed
bedtools sort -i datafiles/COO3/cnv.bed > datafiles/COO3/cnv.sorted.bed

bedtools closest -a datafiles/COO3/sv.sorted.bed -b datafiles/COO3/cnv.sorted.bed -d

<span style="color:purple">[Q]: Which breakpoints from the VCF file are closest to a CNV breakpoint? Do you think that the CNV and the breakpoints are related?

[A]:

4. Annotation of structural variations using gene overlap<a name="Annotation of structural variations using gene overlap"></a>
=============
Now we have identified a set of de novo structural variations in the genome of the patient, the next step is to check which of these variation affect genes in the genome and whether this provides insights into the disease phenotype. Therefore, you will use the bedfile with breakpoints and find out which genes are affected. A file with all human genes and their chromosomal coordinates can be found here: `datafiles/COO3/GenesChromothripsisDNA181.txt`. 

Use a strategy similar to the steps you have taken under (3) to calculate which breakpoints overlap protein coding genes.

<span style="color:purple">[Q]: How many genes are broken by de novo breakpoints in the patient? Please try to use the OMIM database (www.OMIM.org) to find out whether any of the genes has already been linked to human genetics disease.